In [1]:
from llama_index import SimpleDirectoryReader, StorageContext, ServiceContext
from llama_index.indices.vector_store import VectorStoreIndex
from llama_iris import IRISVectorStore

In [3]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv(override=True)

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [18]:
documents = SimpleDirectoryReader("../data/apunts").load_data()
print("Document ID:", documents[0].doc_id)

Document ID: db8290fd-7d79-45e3-a663-0927be342542


In [19]:
username = 'demo'
password = 'demo' 
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

In [21]:
vector_store = IRISVectorStore.from_params(
    connection_string=CONNECTION_STRING,
    table_name="apunts",
    embed_dim=1536,  # openai embedding dimension
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
embed_model = OpenAIEmbedding()
# service_context = ServiceContext.from_defaults(
#     embed_model=embed_model, llm=None
# )

In [22]:
index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context, 
    show_progress=True, 
    # service_context=service_context,
)
query_engine = index.as_query_engine()

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.24it/s]


In [7]:
# # If reconnecting to the vector store, use this: 

# index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# query_engine = index.as_query_engine()

# # Adding documents to existing index

# for d in documents:
#     index.insert(document=d, storage_context=storage_context)

In [26]:
response = query_engine.query("What is RAG?")

In [27]:
import textwrap
print(textwrap.fill(str(response), 100))

RAG is a technique used to base the responses of a Language Model on validated external information.


In [28]:
response = query_engine.query("What happened in the mid 1980s?")
print(textwrap.fill(str(response), 100))

The mid-1980s saw the emergence of a significant technological advancement in the form of Optical
Character Recognition (OCR) technology. This technology played a crucial role in the processing of
PDF documents with tables, as it enabled the extraction of textual information from visual elements
within these documents.


In [30]:
response = query_engine.query("Create questions, and answers, based on the info you know")
print(textwrap.fill(str(response), 100))

What is the purpose of the RAG technique in relation to a LLM? - The RAG technique is used to base
the responses of a LLM on validated external information.  How can contextual integration be applied
in RAG? - Contextual integration in RAG involves loading context into a database, which is then
loaded to be inserted into a prompt.  What are some steps involved in the RAG technique? - Obtaining
textual content from documents, extracting elements, handling metadata such as name and type.  How
are documents normalized in the context provided? - Documents are normalized by converting them into
a common format to identify common elements, enabling consistent preprocessing and reducing
processing costs.  What are some examples of different document formats mentioned in the context? -
JSON, HTML, PowerPoint, and PDF with tables are mentioned as different document formats.  How is
semantic search utilized in the context of document processing? - Semantic search involves using a
database of vec